## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Description
0,0,Trofors,NO,2022-01-24 16:22:25,65.5333,13.4063,33.67,82,100,21.45,light snow
1,1,Saint-Leu,RE,2022-01-24 16:22:25,-21.1500,55.2833,77.86,77,20,18.23,few clouds
2,2,Airai,TL,2022-01-24 16:14:58,-8.9266,125.4092,61.75,89,97,1.88,overcast clouds
3,3,Mayumba,GA,2022-01-24 16:22:26,-3.4320,10.6554,80.60,74,17,7.31,few clouds
4,4,Mao,ES,2022-01-24 16:22:26,39.8885,4.2658,54.16,71,20,11.50,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows. 
preferred_cities_df.notnull().sum()

#Country had one blank 

City_ID             99
City                99
Country             98
Date                99
Lat                 99
Lng                 99
Max Temp            99
Humidity            99
Cloudiness          99
Wind Speed          99
City Description    99
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "City Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,City Description,Lat,Lng,Hotel Name
1,Saint-Leu,RE,77.86,few clouds,-21.1500,55.2833,
3,Mayumba,GA,80.60,few clouds,-3.4320,10.6554,
5,Rikitea,PF,78.10,overcast clouds,-23.1203,-134.9692,
7,Arraial Do Cabo,BR,87.76,clear sky,-22.9661,-42.0278,
12,East London,ZA,78.55,clear sky,-33.0153,27.9116,
15,Vao,NC,77.13,overcast clouds,-22.6667,167.4833,
17,Hermanus,ZA,77.90,scattered clouds,-34.4187,19.2345,
18,Richards Bay,ZA,78.60,clear sky,-28.7830,32.0377,
19,Tautira,PF,78.82,light rain,-17.7333,-149.1500,
25,Butaritari,KI,82.09,broken clouds,3.0707,172.7902,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

City                98
Country             98
Max Temp            98
City Description    98
Lat                 98
Lng                 98
Hotel Name          98
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{City Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))